In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.read.format('parquet').load('abfss://sales@adls7428.dfs.core.windows.net/bronze/products')
df.display()

In [0]:
df.createOrReplaceTempView('products')

In [0]:
spark.sql("""
SELECT * FROM products
""").display()

In [0]:
%sql

create or replace function ete_project.silver.discount_func(p_price double)
returns double
language sql
return p_price*0.90

In [0]:
%sql

select product_id,price,ete_project.silver.discount_func(price) as discounted_price from products;

In [0]:
df = df.withColumn('discounted_price',expr('ete_project.silver.discount_func(price)'))

In [0]:
df.display()

In [0]:
%sql
create or replace function ete_project.silver.upper_funcs(p_brand string)
returns string
language python
as 
$$
  return p_brand.upper()
$$

In [0]:
%sql
select product_id,brand,ete_project.bronze.upper_funcs(brand) as upper_brand from products

In [0]:
df.write.format('delta').mode('append').option('path','abfss://sales@adls7428.dfs.core.windows.net/silver/products').saveAsTable('ete_project.silver.products')